In [1]:
from fontTools.ttLib import TTFont

font = TTFont('SourceHanSans-Regular.otf')
glyph_set = font.getGlyphSet()  # {グリフ名: グリフ} っぽいオブジェクト
cmap = font.getBestCmap()       # {Unicode: グリフ名}

In [2]:
def get_glyph(glyph_set, cmap, char):
    glyph_name = cmap[ord(char)]
    return glyph_set[glyph_name]

In [3]:
L = get_glyph(glyph_set, cmap, 'L')
L

## RecordingPen

In [4]:
from fontTools.pens.recordingPen import RecordingPen

recording_pen = RecordingPen()
L.draw(recording_pen)
recording_pen.value

[('moveTo', ((100, 0),)),
 ('lineTo', ((513, 0),)),
 ('lineTo', ((513, 79),)),
 ('lineTo', ((193, 79),)),
 ('lineTo', ((193, 733),)),
 ('lineTo', ((100, 733),)),
 ('closePath', ())]

In [5]:
い = get_glyph(glyph_set, cmap, 'い')
recording_pen = RecordingPen()
い.draw(recording_pen)
recording_pen.value

[('moveTo', ((226, 696),)),
 ('lineTo', ((130, 698),)),
 ('curveTo', ((135, 674), (136, 633), (136, 610))),
 ('curveTo', ((136, 552), (137, 432), (147, 346))),
 ('curveTo', ((174, 89), (264, -4), (357, -4))),
 ('curveTo', ((425, -4), (486, 53), (545, 221))),
 ('lineTo', ((482, 293),)),
 ('curveTo', ((456, 193), (410, 91), (359, 91))),
 ('curveTo', ((289, 91), (241, 200), (225, 366))),
 ('curveTo', ((218, 447), (217, 538), (218, 600))),
 ('curveTo', ((219, 626), (222, 672), (226, 696))),
 ('closePath', ()),
 ('moveTo', ((742, 669),)),
 ('lineTo', ((664, 642),)),
 ('curveTo', ((758, 526), (818, 330), (835, 152))),
 ('lineTo', ((916, 184),)),
 ('curveTo', ((902, 351), (831, 554), (742, 669))),
 ('closePath', ())]

## SVGPathPen

In [6]:
from fontTools.pens.svgPathPen import SVGPathPen

svg_path_pen = SVGPathPen(glyph_set)
L.draw(svg_path_pen)
svg_path_pen.getCommands()

'M100 0H513V79H193V733H100Z'

In [7]:
with open('L1.svg', 'w') as f:
    f.write('<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 1000 1000">\n')
    f.write(f'    <path d="{svg_path_pen.getCommands()}"/>\n')
    f.write('</svg>\n')

In [8]:
%%html
<img src="L1.svg" style="height: 200px">

In [9]:
with open('L2.svg', 'w') as f:
    f.write('<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 -1000 1000 1000">\n')
    f.write(f'    <path d="{svg_path_pen.getCommands()}" transform="scale(1, -1)"/>\n')
    f.write('</svg>\n')

In [10]:
%%html
<img src="L2.svg" style="height: 200px">

In [11]:
from textwrap import dedent

def save_as_svg(font, char, output_path):
    '''TTFont オブジェクトを受け取り、指定した文字のグリフを SVG として保存する'''
    
    glyph_set = font.getGlyphSet()
    cmap = font.getBestCmap()
    
    # グリフのアウトラインを SVGPathPen でなぞる
    glyph = get_glyph(glyph_set, cmap, char)
    svg_path_pen = SVGPathPen(glyph_set)
    glyph.draw(svg_path_pen)

    # メトリクスを取得
    ascender = font['OS/2'].sTypoAscender
    descender = font['OS/2'].sTypoDescender
    width = glyph.width
    height = ascender - descender
    
    content = dedent(f'''\
        <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 {-ascender} {width} {height}">
            <g transform="scale(1, -1)">
                <!-- ボディの枠 -->
                <rect x="0" y="{descender}" width="{width}" height="{height}"
                    stroke="cyan" stroke-width="2" fill="none"/>
                <!-- グリフ座標系の原点 -->
                <circle cx="0" cy="0" r="8" fill="blue"/>
                <!-- グリフのアウトライン -->
                <path d="{svg_path_pen.getCommands()}"/>
            </g>
        </svg>
    ''')
    
    with open(output_path, 'w') as f:
        f.write(content)

In [12]:
save_as_svg(font, 'L', 'L.svg')
save_as_svg(font, 'い', 'い.svg')

In [13]:
%%html
<img src="L.svg" style="height: 200px">
<img src="い.svg" style="height: 200px">